# MOSMAFS -  Multi-objective vs. Single-objective

### Research questions: 

* _Evolutionary vs. Bayesian Optimization_: How do the proposed methods - the evolutionary and the Bayesian optimization approach - perform in comparison to each other?
* _Effect of Filter Ensembles_: Do the methods benefit from using filter ensembles?
* _Multi-Objective vs. Single-Objective_: Does multi-objective optimization find much sparser solutions without a major loss in predictive performance compared to single-objective optimization? 
* **_Simultaneous Hyperparameter Tuning and Feature Selection_**: Is it beneficial to perform hyperparameter optimization and feature selection simultaneously compared to performing the tasks independently? 

## Setup

In [357]:
# Load packages 
library(data.table)
library(reshape2)
library(mlr)
library(ggplot2)
library(batchtools)
library(gridExtra)
library(ggpubr)
library(ggthemes)
library(plyr)
library(scmamp)
library(xtable)

In [358]:
# stetup for nice plots 
theme_set(theme_pubr())
options(repr.plot.width=15, repr.plot.height=15)

In [359]:
source("../helpers.R")

In [360]:
# path where all results are stored 
respath = "result_data"

In [415]:
# datasets included in the benchmark 
datasets = c("sonar", "ionosphere",
	"hill-valley", "wdbc", "tecator", "lsvt", "isolet", "cnae-9", 
	"semeion", "AP_Breast_Colon", "arcene", 
	"AP_Colon_Kidney", "madelon", "madeline")

In [416]:
tab = lapply(datasets, function(d) {
    task = readRDS(file.path("../data", d, "task.rds"))
    class.dist = task$task.desc$class.distribution
    cr = min(class.dist) / sum(class.dist)
    data.table(id = getTaskId(task), n = getTaskSize(task), p = getTaskNFeats(task), class.ratio = cr)
    })
tab = do.call(rbind, tab)

tab$np.ratio = tab$n / tab$p 
tab = tab[order(p), ]
tab = cbind(1:nrow(tab), tab)

In [417]:
tab

V1,id,n,p,class.ratio,np.ratio
<int>,<chr>,<int>,<int>,<dbl>,<dbl>
1,wdbc,569,30,0.3725835,18.96666667
2,ionosphere,351,33,0.3589744,10.63636364
3,sonar,208,60,0.4663462,3.46666667
4,hill-valley,1212,100,0.4950495,12.12000000
5,tecator,240,124,0.4250000,1.93548387
6,semeion,319,256,0.4952978,1.24609375
7,madeline,3140,259,0.4971338,12.12355212
8,cnae-9,240,282,0.5000000,0.85106383
9,lsvt,126,307,0.3333333,0.41042345


The versions we compare are: 

The versions we compare are: 

|Variant | Description | MO / SO | Internal Name |  |
|---|---|---|---|---|
|GA-MO-FE| GA-based Algorithm | MO | OGHFiFm |   |
|BO-MO-FE | BO-based Algorithm | MO | BSMOF |    |
|BO-SO | BO-based Algorithm | SO |  BS2RF | "State-of-the-art", no feature ensembles  |
|BO-SO-FE | BO-based Algorithm | MO | BS2RF_ENS |  Our best candidate for SO, feature ensembles  |

In [418]:
# Experiments to compare
variants_internal = c("OGHFiFmS", "OIGFiFmS_no_hyperpars500", "BSMOF", "BS2RF_ENS_NJ")
variants = c("GA-MO-FE", "GA-MO-FE-NJ", "BO-MO-FE", "BO-MO-FE-NJ")
names(variants) = variants_internal # for revaluing later 

In [419]:
# read the files 
df = getObjSummaries(datasets = datasets, experiments = variants_internal)
df$variant = revalue(df$variant, variants)

In [420]:
# sanity check if all is read properly
dft = df[gen == 0, ]
grid = setDT(expand.grid(variants, datasets))
names(grid) = c("variant", "problem")

test = dft[, .N, by = c("variant", "problem")]
test = merge(grid, test, by = c("variant", "problem"), all.x = TRUE)

# both should be 0 
test[is.na(N), ]
test[N != 30, ]

variant,problem,N
<chr>,<chr>,<int>


variant,problem,N
<chr>,<chr>,<int>


## Analysis

Comparing Joint vs. non-joint

In [434]:
dfp = df[evals == 2000, ]
dfp = dfp[, .(meandomHV = mean(naive.hout.domHV)), by = c("variant", "learner")]

In [435]:
dfp = reshape2::dcast(dfp, formula = learner ~ variant)
dfp

Using meandomHV as value column: use value.var to override.



learner,BO-MO-FE,BO-MO-FE-NJ,GA-MO-FE,GA-MO-FE-NJ
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
kknn,0.9217127,0.8669721,0.9108019,0.9134438
SVM,0.9330993,0.8718575,0.9240992,0.8891506
xgboost,0.9163586,0.8940340,0.9165383,0.9120815


In [436]:
print(xtable(dfp, type = "latex", digits = 4), file = "viz/RQ4-table-summarized.tex")

In [437]:
dfp = df[evals == 2000, ]
dfp = dfp[, .(meandomHV = mean(naive.hout.domHV)), by = c("variant", "learner", "problem")]
dfp = reshape2::dcast(dfp, formula = learner + problem ~ variant)

Using meandomHV as value column: use value.var to override.



In [438]:
print(xtable(dfp, type = "latex", digits = 4), file = "viz/RQ4-table-detailed.tex")